In [1]:
"""commented out for test execution"""
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


#공용함수

In [2]:
# 수정금지: 타임스탬프용 함수
from datetime import datetime
def printt(*args,**kwargs):
  now = datetime.now()
  now_str = "{:02}:{:02}:{:02}".format(now.hour,now.minute,now.second)
  print(now_str, *args,**kwargs)
  return int(now.hour)*60*60+int(now.minute)*60+int(now.second)

#연관 패키지 설치 (다른작업 금지)

In [3]:
#TODO: 해당 블럭에 패키지 설치하세요.
!pip install attrdict
!pip install transformers
!pip install seqeval
!pip install fastprogress
!pip install konlpy

     |████████████████████████████████| 2.6 MB 8.6 MB/s 
     |████████████████████████████████| 3.3 MB 57.6 MB/s 
     |████████████████████████████████| 636 kB 61.9 MB/s 
     |████████████████████████████████| 895 kB 61.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 43 kB 1.6 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16181 sha256=ca2705febfe8c016a83901cbe6fe5378ecfbd19748d4ac49ffa391def5e54e8a
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval
     |████████████████████████████████| 19.4 MB 1.1 MB/s 
     |████████████████████████████████| 448 kB 72.5 MB/s 
     |████████████████████████████████| 86 kB 8.1 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstall

# 파일로딩 (다른작업 금지)

In [4]:
#TODO: 해당 블럭에 필요 파일 로딩 코드 넣으시오.
from transformers import (
    ElectraConfig,
    ElectraTokenizer,
    ElectraForSequenceClassification,
)
train_file = '/content/drive/MyDrive/v1/train.txt'
test_file = '/content/drive/MyDrive/v1/evaluation.txt'
pretrain_bin = '/content/drive/MyDrive/pretrained_model/koelectra_pretrained.bin'
tokenizer = ElectraTokenizer.from_pretrained(
    '/content/drive/MyDrive/pretrained_model/', do_lower_case = False)
config_1 = ElectraConfig.from_pretrained('/content/drive/MyDrive/pretrained_model',
                                       num_labels=1)
config_2 = ElectraConfig.from_pretrained('/content/drive/MyDrive/pretrained_model',
                                       num_labels=16)
lines_dict = {}
import torch

with open(train_file, "r", encoding="utf-8") as f:
    lines = []
    for line in f:
        lines.append(line.strip())
    lines_dict['train'] = lines
with open(test_file, "r", encoding="utf-8") as f:
    lines = []
    for line in f:
        lines.append(line.strip())
    lines_dict['test'] = lines

# 모델 학습

In [5]:
# 수정금지: 타임스탬프
_model_build_start_time = printt("Model building: Start")
_model_build_start_time

10:09:00 Model building: Start


36540

In [13]:
#TODO: 블럭에 모델 학습 - 빌딩 코드를 넣으세요. (시간측정 구간)

# set config args for classification
from transformers import (
    ElectraConfig,
    ElectraTokenizer,
    ElectraForSequenceClassification,
)
import argparse
import json
import shutil
import os
import glob
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from transformers import (
    AdamW,
    get_linear_schedule_with_warmup
)
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from fastprogress.fastprogress import master_bar, progress_bar
from attrdict import AttrDict

lines_dict['input'] = []
args = AttrDict(
    {
        'version': 1,
        'data_dir': '',
        'train_file': lines_dict['train'],
        'test_file': lines_dict['test'],
        # 'val_file': 'test.txt',
        'input_file': lines_dict['input'],
        'task': 'classification',
        'config_1': config_1,
        'config_2': config_2,
        'tokenizer': ElectraTokenizer,
        'model': ElectraForSequenceClassification,
        'device': 'cuda',
        'do_lower_case': False,  
        'max_seq_len': 64, 
        'num_train_epochs': 10, 
        'weight_decay_1': 0.3, 
        'weight_decay_2': 0.2,
        'gradient_accumulation_steps': 1, 
        'adam_epsilon': 1e-08, 
        'warmup_proportion_1': 0.0, 
        'warmup_proportion_2': 0.0,
        'max_steps': -1, 
        'max_grad_norm': 1.0, 
        'no_cuda': False, 
        'tokenizer': tokenizer,
        'model_path': '/content/drive/MyDrive/pretrained_model/',
        'output_dir_1': 'checkpoints_sentiment', 
        'output_dir_2': 'checkpoints_classification', 
        'seed': 42, 
        'train_batch_size': 16, 
        'eval_batch_size': 16, 
        'learning_rate_1': 5e-06,
        'learning_rate_2': 5e-05,
     }
)

# make Input Example
class InputExample(object):
    """
    A single training/test example for simple sequence classification.
    """

    def __init__(self, review_token, label1, label2):
        self.review_token = review_token
        self.label1 = label1
        self.label2 = label2

    def __repr__(self):
        return str(self.to_json_string())

    def to_dict(self):
        """Serializes this instance to a Python dictionary."""
        output = copy.deepcopy(self.__dict__)
        return output

    def to_json_string(self):
        """Serializes this instance to a JSON string."""
        return json.dumps(self.to_dict(), indent=2, sort_keys=True) + "\n"
        
# make Input Feature
class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, attention_mask, token_type_ids, label1, label2):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.token_type_ids = token_type_ids
        self.label1 = label1
        self.label2 = label2

    def __repr__(self):
        return str(self.to_json_string())

    def to_dict(self):
        """Serializes this instance to a Python dictionary."""
        output = copy.deepcopy(self.__dict__)
        return output

    def to_json_string(self):
        """Serializes this instance to a JSON string."""
        return json.dumps(self.to_dict(), indent=2, sort_keys=True) + "\n"

from torch.utils.data import TensorDataset

def load_dataset(args, tokenizer, mode):
    features = get_features(args, tokenizer, mode)

    # Convert to Tensors and build dataset
    all_input_ids = torch.tensor(
        [f.input_ids for f in features], 
        dtype=torch.long)
    all_attention_mask = torch.tensor(
        [f.attention_mask for f in features],
         dtype=torch.long)
    all_token_type_ids = torch.tensor(
        [f.token_type_ids for f in features],
        dtype=torch.long)

    all_labels_1 = torch.tensor([f.label1 for f in features],
                        dtype=torch.float)
    all_labels_2 = torch.tensor([f.label2 for f in features],
                              dtype=torch.long)

    dataset = TensorDataset(all_input_ids,
                            all_attention_mask,
                            all_token_type_ids,
                            all_labels_1,
                            all_labels_2)
    return dataset


def get_features(args, tokenizer, mode):
    # load data and labels using Processor
    processor = Processor(args)
    try:
        examples = processor.get_examples(mode)
    except ValueError:
        print('possible modes: train, val, test')

    # if args.task == 'classification':
    
    labels1 = [float(example.label1) for example in examples]
    labels2 = [int(example.label2) for example in examples]


    batch_encoding = tokenizer.batch_encode_plus(
        [(example.review_token) for example in examples],
        max_length=args.max_seq_len,
        padding="max_length",
        add_special_tokens=True,
        truncation=True,
    )

    features = []
    for i in range(len(examples)):
        inputs = {k: batch_encoding[k][i] for k in batch_encoding}
        
        # For xlm-roberta
        inputs["token_type_ids"] = [0] * len(inputs["input_ids"])  
        feature = InputFeatures(**inputs, label1=labels1[i], label2=labels2[i])
        features.append(feature)
        

    return features


class Processor(object):
    def __init__(self, args):
        self.args = args


    def _read_file(cls, mode):
        """Reads a tab separated value file."""
        return lines_dict[mode]

    def _create_examples(self, lines, set_type):
        """Creates examples for the training and dev sets."""
        examples = []
        for (i, line) in enumerate(lines[0:]):
            line = line.split("\t")
            review_token = line[0]
            if not set_type == 'input':
                label1 = line[1]
                label2 = line[2]
            else:
                label1 = 0
                label2 = 0
            examples.append(InputExample(review_token=review_token, label1=label1
                                         , label2=label2))
        return examples

    def get_examples(self, mode):
        """
        Args:
            mode: train, test
        """
        file_to_read = None
        if mode == "train":
            file_to_read = self.args.train_file
        elif mode == "test":
            file_to_read = self.args.test_file
        elif mode == 'input':
            file_to_read = self.args.input_file

        return self._create_examples(
            self._read_file(mode), mode
        )


# TODO: acc. calculating logic impl. + figure adding logic impl.
        
def train(args, datasets):
    train_sampler = RandomSampler(datasets['train'])
    train_loader = DataLoader(datasets['train'],
                              sampler=train_sampler,
                              batch_size=args.train_batch_size)
    val_sampler = RandomSampler(datasets['val'])
    val_loader = DataLoader(datasets['val'],
                            sampler=val_sampler,
                            batch_size=args.eval_batch_size)
    dataloaders = {'train': train_loader, 'val': val_loader}
    
    t_total = (
        len(dataloaders['train']) // args.gradient_accumulation_steps
        * args.num_train_epochs
    )

    # Load models <model_1: sentiment, model_2: classifiation>
    config_1 = args.config_1
    model_1 = args.model.from_pretrained(pretrain_bin,
                                         config=config_1).to(args.device)
    config_2 = args.config_2
    model_2 = args.model.from_pretrained(pretrain_bin,
                                         config=config_2).to(args.device)

   
    # Prepare optimizer and schedule (linear warmup and decay)
    no_decay = ['bias', 'LayerNorm.weight']
    optimizer_grouped_parameters_1 = [
        {
            'params': [p for n, p in model_1.named_parameters()
                    if not any(nd in n for nd in no_decay)],
            'weight_decay': args.weight_decay_1
         },
        {
            'params': [p for n, p in model_1.named_parameters()
                    if any(nd in n for nd in no_decay)],
            'weight_decay': 0.0
         }
    ]
    
    optimizer_grouped_parameters_2 = [
        {
            'params': [p for n, p in model_2.named_parameters()
                    if not any(nd in n for nd in no_decay)],
            'weight_decay': args.weight_decay_2
         },
        {
            'params': [p for n, p in model_2.named_parameters()
                    if any(nd in n for nd in no_decay)],
            'weight_decay': 0.0
         }
    ]
    optimizer_1 = AdamW(optimizer_grouped_parameters_1,
                      lr=args.learning_rate_1,
                      eps=args.adam_epsilon)
    optimizer_2 = AdamW(optimizer_grouped_parameters_2,
                      lr=args.learning_rate_2,
                      eps=args.adam_epsilon)
    scheduler_1 = get_linear_schedule_with_warmup(
        optimizer_1,
        num_warmup_steps=int(t_total * args.warmup_proportion_1),
        num_training_steps=t_total
        )
    scheduler_2 = get_linear_schedule_with_warmup(
        optimizer_2,
        num_warmup_steps=int(t_total * args.warmup_proportion_2),
        num_training_steps=t_total
        )

    if os.path.isfile(os.path.join(args.model_path, "optimizer.pt")) \
    and os.path.isfile(os.path.join(args.model_path, "scheduler.pt")):
        # Load optimizer and scheduler states
        optimizer_1.load_state_dict(
            torch.load(os.path.join(args.model_path, "optimizer.pt"))
            )
        optimizer_2.load_state_dict(
            torch.load(os.path.join(args.model_path, "optimizer.pt"))
            )
        scheduler_1.load_state_dict(
            torch.load(os.path.join(args.model_path, "scheduler.pt"))
            )
        scheduler_2.load_state_dict(
            torch.load(os.path.join(args.model_path, "scheduler.pt"))
            )

    print(f'training started <model_1: sentiment, model_2: classifiation>')
    global_step = 0
    top_acc_1 = 0.
    top_acc_2 = 0.
    model_1.zero_grad()
    model_2.zero_grad()
    mb = master_bar(range(int(args.num_train_epochs)))
    for epoch in mb:
        mb.main_bar.comment = f'training in progress: epoch {epoch+1}'
        accuracies_1, accuracies_2, losses = {}, {}, {}
        for stage, dataloader in dataloaders.items():
            epoch_iterator = progress_bar(dataloader, parent=mb)
            num_correct_1, num_correct_2, num_items = 0, 0, 0
            running_loss = 0
            if stage == 'train':
                model_1.train()
                model_2.train()  
            else:
                model_1.eval()
                model_2.eval()          
            for step, batch in enumerate(epoch_iterator):
                mb.child.comment = f'epoch {epoch+1} {stage}'
                if stage == 'train':
                    optimizer_1.zero_grad() # why is this not in original func?
                    optimizer_2.zero_grad()
                batch = tuple(t.to(args.device) for t in batch)
                input_ids = batch[0]
                attention_mask = batch[1]
                token_type_ids = batch[2]
                labels_1 = batch[3]
                labels_2 = batch[4]

                inputs_1 = {
                    "input_ids": input_ids,
                    "attention_mask": attention_mask,
                    "token_type_ids": token_type_ids,
                    "labels": labels_1
                }

                inputs_2 = {
                    "input_ids": input_ids,
                    "attention_mask": attention_mask,
                    "token_type_ids": token_type_ids,
                    "labels": labels_2
                }
                # model output: [loss, grad_fn (MSE), logits, batches]
                outputs_1 = model_1(**inputs_1)
                outputs_2 = model_2(**inputs_2)

                loss_1 = outputs_1[0]
                preds_1 = torch.round(outputs_1.logits).view(-1)
                num_items += preds_1.shape[0]
                num_correct_1 += (labels_1 == preds_1).sum()                
                
                loss_2 = outputs_2[0]                
                preds_2 = torch.argmax(outputs_2.logits, dim=1)
                num_correct_2 += (labels_2 == preds_2).sum()

                # loss_1, loss_2 = loss_1 + loss_2, loss_1 + loss_2

                if stage == 'train':
                    loss_1.backward()
                    loss_2.backward()
                    torch.nn.utils.clip_grad_norm_(model_1.parameters(),
                                                   args.max_grad_norm)
                    torch.nn.utils.clip_grad_norm_(model_2.parameters(),
                                                   args.max_grad_norm)
                    optimizer_1.step()
                    optimizer_2.step()
                    scheduler_1.step()
                    scheduler_2.step()

                    model_1.zero_grad()
                    model_2.zero_grad()
                    global_step += 1
                running_loss += loss_1.item()
                    

            accuracies_1[stage] = num_correct_1 / num_items
            accuracies_2[stage] = num_correct_2 / num_items
            losses[stage] = running_loss / len(dataloader)
            
        # Save model checkpoint
        if top_acc_1 < accuracies_1['val']:
            if os.path.exists(args.output_dir_1):
                shutil.rmtree(args.output_dir_1)
            output_dir_1 = os.path.join(args.output_dir_1,
                            "1-checkpoint-{}".format(epoch + 1))
            if not os.path.exists(output_dir_1):
                os.makedirs(output_dir_1)
            model_1_to_save = (
                model_1.module if hasattr(model_1, "module") else model_1
            )    
            model_1_to_save.save_pretrained(output_dir_1)
            torch.save(args, os.path.join(args.output_dir_1, "training_args.bin"))
            top_acc_1 = accuracies_1['val']

        if top_acc_2 < accuracies_2['val']:
            # Save model checkpoint
            if os.path.exists(args.output_dir_2):
                shutil.rmtree(args.output_dir_2)
            output_dir_2 = os.path.join(args.output_dir_2,
                                        "2-checkpoint-{}".format(epoch + 1))
            if not os.path.exists(output_dir_2):
                os.makedirs(output_dir_2)
            model_2_to_save = (
                model_2.module if hasattr(model_2, "module") else model_2
            )
            model_2_to_save.save_pretrained(output_dir_2)
            torch.save(args, os.path.join(args.output_dir_2, "training_args.bin"))
            top_acc_2 = accuracies_2['val']

        print(accuracies_1, accuracies_2)
        if args.max_steps > 0 and global_step > args.max_steps:
            del model_1, model_2
            torch.cuda.empty_cache()
            break

    return output_dir_1, output_dir_2




torch.cuda.manual_seed_all(args.seed)

# tokenizer = args.tokenizer.from_pretrained(
#     args.model_name_or_path,
#     do_lower_case=args.do_lower_case
# )

train_dataset = load_dataset(args, tokenizer, mode="train")
val_dataset = load_dataset(args, tokenizer, mode="test")
datasets = {'train': train_dataset, 'val': val_dataset}

# torch.cuda.empty_cache()
# torch.cuda.ipc_collect()
# torch.cuda.memory_summary(abbreviated=True)


# Train
train(args, datasets)

Some weights of the model checkpoint at /content/drive/MyDrive/pretrained_model/koelectra_pretrained.bin were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/pretrained_model/koel

KeyboardInterrupt: ignored

In [8]:
# 수정금지: 타임스탬프
_model_build_end_time = printt("Model building: Start")
print(_model_build_end_time - _model_build_start_time)

10:14:12 Model building: Start
312


#모델 분류

In [9]:
# 수정금지: 타임스탬프
_test_start_time = printt("TEST: Start")

10:14:14 TEST: Start


In [15]:
#TODO: 해당 블럭에 테스트 수행을 위한 코드를 넣으세요. (시간측정 구간)
#분석 파일은 tsv 파일로 제공되며, 제공되는 학습데이터 파일과 동일한 레이아웃(단, 정답 분류 =공백)으로 제공됩니다.

from konlpy.tag import Kkma
import re

def test(args, test_data, mode, global_step=None):
  
    checkpoints_1 = list(
                os.path.dirname(c) for c in
                sorted(glob.glob(args.output_dir_1 + "/**/" + "pytorch_model.bin", recursive=True))
            )
    checkpoints_2 = list(
                os.path.dirname(c) for c in
                sorted(glob.glob( args.output_dir_2 + "/**/" + "pytorch_model.bin", recursive=True))
            )
    print(checkpoints_1, checkpoints_2)
    checkpoint_1 = checkpoints_1[-1]
    checkpoint_2 = checkpoints_2[-1]

    model_1 = args.model.from_pretrained(checkpoint_1).to(args.device)
    model_2 = args.model.from_pretrained(checkpoint_2).to(args.device)

    test_sampler = SequentialSampler(test_data)
    test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=args.eval_batch_size)
    num_correct_1, num_correct_2, num_items = 0, 0, 0
    df_result_1 = pd.DataFrame(columns=['Review', 'Label1', 'Label2', 'Preds'])
    df_result_2 = pd.DataFrame(columns=['Review', 'Label1', 'Label2', 'Preds'])
    id_list = []
    preds_1_list = []
    preds_2_list = []
    out_labels_1 = []
    out_labels_2 = []
    for batch in progress_bar(test_dataloader):
        preds_temp = None
        model_1.eval()
        model_2.eval()
        batch = tuple(t.to(args.device) for t in batch)

        with torch.no_grad():
            inputs = {
                "input_ids": batch[0],
                "attention_mask": batch[1],
                "token_type_ids": batch[2],
            }     
            outputs_1 = model_1(**inputs)
            outputs_2 = model_2(**inputs)
            preds_1 = torch.round(outputs_1.logits).view(-1)
            preds_2 = torch.argmax(outputs_2.logits, dim=1)
            preds_1_list += list(preds_1.detach().cpu().numpy().astype(int))
            preds_2_list += list(preds_2.detach().cpu().numpy())


    return preds_1_list, preds_2_list

def complete_result(list_preds):
    label_dict_classification = {'중립': 0, '상담원': 1, '상담시스템': 2, '혜택': 3, '할부금융상품': 4,
                  '카드상품': 5, '청구입금': 6, '심사/한도': 7, '생활편의서비스': 8,
                  '상담/채널': 9, '리스렌탈상품': 10, '라이프서비스': 11, '금융상품': 12,
                  '고객정보관리': 13, '삼성카드': 14, '기타': 15}
    label_dict_sentiment = {'칭찬': 0, '불만': 1, '': -1}
    label_dict_classification = dict((v, k) for k, v in label_dict_classification.items())
    label_dict_sentiment = dict((v, k) for k, v in label_dict_sentiment.items())

    for i in range(len(list_preds[1])):
        if list_preds[1][i] == 0:
            list_preds[0][i] = -1
    for i in label_dict_classification:
      if i not in [0, 1, 2, 15]:
        label_dict_classification[i] = '>삼성카드>' + label_dict_classification[i]
      else:
        if i == 1 or i == 2:
          label_dict_classification[i] = '>고객서비스>' + label_dict_classification[i]
        elif i == 15:
          label_dict_classification[i] = '>' + label_dict_classification[i]
    sent_result = np.vectorize(label_dict_sentiment.get)(list_preds[0])
    class_result = np.vectorize(label_dict_classification.get)(list_preds[1])
    result_arr = np.core.defchararray.add(sent_result, class_result)
    list_output = list(result_arr)
    return list_output


kkma = Kkma()
# input_xlsx = '/content/drive/MyDrive/input_result.xlsx'
xls = pd.ExcelFile('/content/drive/MyDrive/input_data.xlsx')
test_data = pd.read_excel(xls, '학습데이터')
df = pd.read_excel(xls, '학습데이터', usecols='C')
df_temp = pd.DataFrame(columns=['Review'])
for sentence in test_data['TEXT']:
    sentence = re.sub('[^가-힣a-zA-Z0-9]', "", sentence)
    review = '[SEP]'.join(kkma.sentences(sentence))
    if [review] not in df.values[:, 0:1]:
        df_temp = df.append(pd.DataFrame({'Review': review}, index=[id]))

df_temp.to_csv('input_data.txt', header=None, index=None, sep='\t', mode='a')
with open('input_data.txt', "r", encoding="utf-8") as f:
    lines = []
    for line in f:
        lines.append(line.strip())
    lines_dict['input'] = lines
input_dataset = load_dataset(args, tokenizer, mode="input")

list_preds = []
preds_1, preds_2 = test(args, input_dataset, 'input')
list_preds.append(preds_1)
list_preds.append(preds_2)
list_output = complete_result(list_preds)



['checkpoints_sentiment/1-checkpoint-1'] ['checkpoints_classification/2-checkpoint-1']


In [16]:
# 수정금지: 타임스탬프
_test_end_time = printt("Model building: Start")
print(_test_end_time - _test_start_time)

10:25:13 Model building: Start
659


# 결과출력

In [19]:
#TODO:해당 블럭에 테스트 결과를 파일로 저장하는 코드를 넣으세요. (시간측정 제외)
#저장 파일은tsv 파일로 제공되는 학습데이터 파일과 동일한 레이아웃(단, 정답 분류 = 테스트 결과 도출된 양식)으로 저장

import openpyxl
myworkbook=openpyxl.load_workbook('/content/drive/MyDrive/input_data.xlsx')
worksheet= myworkbook.get_sheet_by_name('학습데이터')
for i in range(len(list_output)):
    worksheet[f'D{i+2}']=list_output[i]
worksheet['D1'] = 'INT'
myworkbook.save(filename = '/content/drive/MyDrive/result_data.xlsx')


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  
